In [3]:
import os
os.chdir("../../..")

In [4]:


from dragon.experiments.monash_archive.dataset import gluonts_dataset
from dragon.experiments.monash_archive.datasets_configs import m1_monthly_config

train_ds, test_ds, config = gluonts_dataset(m1_monthly_config)


2023-10-12 17:32:00,598 | INFO | Started loading m1_monthly


TypeError: __new__() got an unexpected keyword argument 'freq'